In [92]:
import requests
from bs4 import BeautifulSoup as bs
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import csv
import emoji

In [2]:
movies_urls = ['https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film.html', 
               'https://turkcealtyazi.org/ta250.html']
domain = 'https://turkcealtyazi.org'

In [3]:
chromedriver_path = '/usr/local/bin/chromedriver'

In [4]:
options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--verbose")

In [48]:
driver = webdriver.Chrome(options=options)

In [5]:
def extract_movie_data(row):
    columns = row.find_all('td')
    
    title_link = columns[2].find('a')
    title = title_link.find('span').text.strip()
    movie_path = title_link['href']
    
    title_text = columns[2].text
    year = re.search(r'\((\d{4})\)', title_text).group(1)
    rating = columns[3].find('strong').text.strip()
    
    return {
        'movie_name': title,
        'year': year,
        'rating': rating,
        'link': f'{domain}{movie_path}'
    }

In [13]:
def get_movies_from_link(movies_url):
    movies = []
    while True:
        old_url = movies_url
        driver.get(movies_url)
        html = driver.page_source
        soup = bs(html, 'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr')
        del rows[0]
        
        for row in rows:
            movie = extract_movie_data(row)
            movies.append(movie)

        pagination = soup.find('div', {'class': 'pagin'})
        if pagination:
            pages_a = pagination.find_all('a')
            if pages_a:
                pages_a = pages_a[-1]
                
                if pages_a and hasattr(pages_a, 'text') and 'Sonraki' in pages_a.text:
                    next_page = f'{domain}{pages_a["href"]}'
                    print(f'Next page: {next_page}')
                    time.sleep(1)
                    movies_url = next_page
        
        if old_url == movies_url:
            break
            
    return movies
    

In [14]:
movies = []

In [15]:
for movies_url in movies_urls:
    movies.extend(get_movies_from_link(movies_url))

Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-2.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-3.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-4.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-5.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-6.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-7.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-8.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-9.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-10.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-11.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-film-12.html
Next page: https://turkcealtyazi.org/olmeden-once-gormeniz-gereken-1001-f

In [16]:
len(movies)

1250

In [19]:
def extract_movie_genre(movie):
    driver.get(movie['link'])
    html = driver.page_source
    soup = bs(html, 'html.parser')
    genres = soup.find_all('span', {'itemprop': 'genre'})
    genres = [genre.text for genre in genres] if genres else None
    return genres[0] if genres else None

In [40]:
def extract_comments(movie):
    comment_url = movie['link'].replace('mov', 'yorumlar')
    comments = []
    while True:
        old_url = comment_url
        
        driver.get(comment_url)
        html = driver.page_source
        soup = bs(html, 'html.parser')
        comment_divs = soup.find_all('div', {'class': 'nyorumdivv'})
        if comment_divs:
            for comment in comment_divs:
                text = rating = None
                body = comment.find('div', {'itemprop': 'reviewBody'})
                if body:
                    text = body.text
                
                rating_span = comment.find('span', {'class': 'yorum-oy'})
                if rating_span:
                    rating = rating_span.text
                    
                comments.append({
                    'text': text,
                    'rating': rating
                })
        pagination = soup.find('div', {'class': 'pagin'})
        if pagination:
            pages_a = pagination.find_all('a')
            if pages_a:
                pages_a = pages_a[-1]
                
                if pages_a and hasattr(pages_a, 'text') and 'Sonraki' in pages_a.text:
                    next_page = f'{domain}{pages_a["href"]}'
                    comment_url = next_page
                    
        if old_url == comment_url:
            break
            
    return comments

In [49]:
def get_movies_data(movies):
    result = []
    i = 0
    for movie in movies:
        i += 1
        print(movie['movie_name'])
        print(f'{i}/{len(movies)}')
        genre = extract_movie_genre(movie)
        comments = extract_comments(movie)
        
        result.append([
            {'movie_name': movie['movie_name'], 'year': movie['year'], 'rating': movie['rating'], 
             'genre': genre, 'comment_text': c['text'], 'comment_rating': int(c['rating']) if c['rating'] else None} for c in comments
        ])
        
        time.sleep(1)

    return result

In [50]:
movies_data = get_movies_data(movies)

Aya Seyahat
1/1250
https://turkcealtyazi.org/mov/0000417/a-trip-to-the-moon.html
Büyük Tren Soygunu
2/1250
https://turkcealtyazi.org/mov/0000439/the-great-train-robbery.html
Bir Ulusun Doğuşu
3/1250
https://turkcealtyazi.org/mov/0004972/the-birth-of-a-nation.html
Vampirler
4/1250
https://turkcealtyazi.org/mov/0006206/les-vampires.html
Hoşgörüsüzlük
5/1250
https://turkcealtyazi.org/mov/0006864/intolerance-loves-struggle-throughout-the-ages.html
Dr. Caligari'nin Muayenehanesi
6/1250
https://turkcealtyazi.org/mov/0010323/das-cabinet-des-dr-caligari.html
Kırık Tomurcuklar
7/1250
https://turkcealtyazi.org/mov/0009968/broken-blossoms-or-the-yellow-man-and-the-girl.html
Doğu Yolu
8/1250
https://turkcealtyazi.org/mov/0011841/way-down-east.html
Within Our Gates
9/1250
https://turkcealtyazi.org/mov/0011870/within-our-gates.html
Hayalet Araba
10/1250
https://turkcealtyazi.org/mov/0012364/korkarlen.html
Fırtına Çocukları
11/1250
https://turkcealtyazi.org/mov/0012532/orphans-of-the-storm.html
Madam

In [54]:
extended_movies_data = []
for data in movies_data:
    for d in data:
        extended_movies_data.append(d)

In [61]:
len(extended_movies_data)

29105

In [62]:
extended_movies_data[0]

{'movie_name': 'Aya Seyahat',
 'year': '1902',
 'rating': '8.1',
 'genre': 'Kısa Metraj',
 'comment_text': 'İnsanlık için küçük, sinema için dev bir adım!',
 'comment_rating': 8}

In [159]:
movies_data[0]

{'movie_name': 'Aya Seyahat',
 'year': '1902',
 'rating': '8.1',
 'link': 'https://turkcealtyazi.org/mov/0000417/a-trip-to-the-moon.html',
 'genres': ['Kısa Metraj', 'Macera', 'Fantastik', 'Bilim-Kurgu'],
 'comments': ['İnsanlık için küçük, sinema için dev bir adım!',
  'Ay\'a seyahat hakkında düşünüldüğünde, insanın aklına hemen sinemanın ilk dönemlerine ilişkin o özgün ve efsanevi anlayış gelir: "Kuralları" tam da üretim aşamasında belirlenen bir sanattı o zamanlar sinema. 1902 yılında gösterime giren bu Fransız filmi, süresi açısından (yaklaşık 14 dakika) geçen yüzyılın başında çekilen genelde iki dakikalık kısa filmlere kıyasla o zaman için bir devrim niteliğindeydi. \n\nSürprizbozan: Göster\n\nAy\'a seyahat, tiyatro oyunculuğu ve sihirbazlık geçmişinin etkileriyle filmi yapan yönetmen Georgies Melies\'in sahne adamı kişiliğini doğrudan yansıtır. Filmde daha sonra yaygın bir şekilde kullanılacak bindirme, zincirleme ve benzeri kurgu teknikleri gibi bir çok ünlü film tekniği cesurca

In [65]:
with open('movie_data_comment.csv', 'w+', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=extended_movies_data[0].keys())
    writer.writeheader()
    writer.writerows(extended_movies_data)

In [120]:
df = pd.read_csv('movie_data_comment.csv')

In [121]:
df.head()

,movie_name,year,rating,genre,comment_text,comment_rating
0,Aya Seyahat,1902,8.1,Kısa Metraj,"İnsanlık için küçük, sinema için dev bir adım!",8.0
1,Aya Seyahat,1902,8.1,Kısa Metraj,"Ay'a seyahat hakkında düşünüldüğünde, insanın ...",9.0
2,Aya Seyahat,1902,8.1,Kısa Metraj,Film ile ilgili yazıda yönetmen tarafından eğl...,9.0
3,Aya Seyahat,1902,8.1,Kısa Metraj,Aslında 1 asırı devirmiş olan bu film hakkında...,8.0
4,Aya Seyahat,1902,8.1,Kısa Metraj,Hayal gücü ile olağanüstülüğün karıştığı bir f...,6.0


In [122]:
empty_comments = df[df['comment_text'].isin(['', "", None])]

In [123]:
empty_comments.shape

(0, 6)

In [124]:
empty_comment_ratings = empty_comments[empty_comments['comment_rating'].isnull()]

In [125]:
empty_comment_ratings.shape

(0, 6)

In [126]:
unique_genres = df['genre'].unique()

In [127]:
unique_genres.shape

(19,)

In [128]:
df.value_counts(df['genre'])

genre
Dram           8070
Aksiyon        5619
Suç            3871
Komedi         3175
Macera         2837
Biyografi      2449
Animasyon      1048
Gizem           508
Korku           423
Belgesel        414
Western         345
Kısa Metraj     122
Bilim-Kurgu      64
Kara Film        45
Fantastik        39
Gerilim          19
Romantik          2
Savaş             2
Name: count, dtype: int64

In [130]:
duplicate_comments = df[df.duplicated(subset=['movie_name', 'comment_text', 'comment_rating'])]

In [131]:
duplicate_comments.shape

(5478, 6)

In [132]:
duplicate_comments.head()

,movie_name,year,rating,genre,comment_text,comment_rating
28,Aya Seyahat,1902,8.1,Kısa Metraj,"İnsanlık için küçük, sinema için dev bir adım!",8.0
32,Aya Seyahat,1902,8.1,Kısa Metraj,Film ile ilgili yazıda yönetmen tarafından eğl...,9.0
33,Aya Seyahat,1902,8.1,Kısa Metraj,"Ay'a seyahat hakkında düşünüldüğünde, insanın ...",9.0
152,Altına Hücum,1925,8.1,Macera,Kim ne derse desin bu film gerçekten kendi zam...,8.0
153,Altına Hücum,1925,8.1,Macera,"Aklınızdakileri, yaşadıklarınızı bir kenara bı...",10.0


In [133]:
unique_comments = df.drop_duplicates(subset=['movie_name', 'comment_text', 'comment_rating'])

In [134]:
unique_comments.shape

(23627, 6)

In [135]:
unique_comments.head()

,movie_name,year,rating,genre,comment_text,comment_rating
0,Aya Seyahat,1902,8.1,Kısa Metraj,"İnsanlık için küçük, sinema için dev bir adım!",8.0
1,Aya Seyahat,1902,8.1,Kısa Metraj,"Ay'a seyahat hakkında düşünüldüğünde, insanın ...",9.0
2,Aya Seyahat,1902,8.1,Kısa Metraj,Film ile ilgili yazıda yönetmen tarafından eğl...,9.0
3,Aya Seyahat,1902,8.1,Kısa Metraj,Aslında 1 asırı devirmiş olan bu film hakkında...,8.0
4,Aya Seyahat,1902,8.1,Kısa Metraj,Hayal gücü ile olağanüstülüğün karıştığı bir f...,6.0


In [136]:
unique_comments.value_counts(unique_comments['comment_rating'])

comment_rating
10.0    5423
9.0     4775
8.0     4390
7.0     2264
6.0      998
5.0      515
1.0      298
4.0      258
3.0      154
2.0       96
Name: count, dtype: int64

In [147]:
def clean_comment(text):
    if isinstance(text, str):
        text = text.lower()
        emoji.replace_emoji(text, '')
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'(.)\1{2,}', r'\1', text)
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        return text
    
    return text

In [148]:
unique_comments.loc[:, 'comment_text'] = unique_comments['comment_text'].astype(str).fillna('')

In [149]:
unique_comments.loc[:, 'comment_text'] = unique_comments['comment_text'].apply(clean_comment)

In [140]:
unique_comments.value_counts(unique_comments['comment_rating'])

comment_rating
10.0    5423
9.0     4775
8.0     4390
7.0     2264
6.0      998
5.0      515
1.0      298
4.0      258
3.0      154
2.0       96
Name: count, dtype: int64

In [141]:
mean_ratings = unique_comments.groupby('movie_name')['comment_rating'].mean()

In [142]:
# replace nan values with mean ratings
unique_comments.loc[:, 'comment_rating'] = unique_comments.apply(lambda x: mean_ratings[x['movie_name']] if pd.isnull(x['comment_rating']) else x['comment_rating'], axis=1)

In [143]:
movie_mean_ratings = unique_comments.groupby('movie_name')['rating'].mean()

In [144]:
unique_comments.loc[:, 'comment_rating'] = unique_comments.apply(lambda x: movie_mean_ratings[x['movie_name']] if pd.isnull(x['comment_rating']) else x['comment_rating'], axis=1)

In [145]:
unique_comments[unique_comments['comment_rating'].isnull()]

,movie_name,year,rating,genre,comment_text,comment_rating


In [150]:
unique_comments.to_csv('cleaned_movie_data_comment.csv', index=False)